In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import pearsonr
from collections import defaultdict
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
import random
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor
import os
from sklearn.base import clone

# --- Configuração dos modelos ---
from tensorflow.keras.applications.xception import Xception, preprocess_input as preprocess_xception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_vgg19
from tensorflow.keras.applications.resnet import ResNet50, ResNet101, ResNet152, preprocess_input as preprocess_resnet
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input as preprocess_resnet_v2
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_inception_v3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input as preprocess_inception_resnet_v2
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as preprocess_mobilenet_v2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201, preprocess_input as preprocess_densenet
from tensorflow.keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input as preprocess_nasnet
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7, preprocess_input as preprocess_efficientnet

model_input_sizes = {model: (1024, 1024) for model in ['ResNet50']}

model_info = {'ResNet50': (ResNet50, preprocess_resnet)}

# Aumentadores globais
data_augmenters = [
    ImageDataGenerator(horizontal_flip=True),
    ImageDataGenerator(brightness_range=[0.8, 1.2]),
    ImageDataGenerator(preprocessing_function=lambda x: x + np.random.normal(0, 5, x.shape))
]

# --- Funções auxiliares ---
def augment_and_extract(model, img_path, model_name):
    img = load_img(img_path, target_size=model_input_sizes[model_name])
    augmenter = random.choice(data_augmenters)
    aug_img = augment_image(img, augmenter)
    aug_img_array = img_to_array(aug_img)
    aug_img_array = np.expand_dims(aug_img_array, axis=0)
    preprocessed_aug_img = model_info[model_name][1](aug_img_array)
    feats = model.predict(preprocessed_aug_img, verbose=0)
    return feats.flatten()

def augment_image(img, augmenter):
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    aug_iter = augmenter.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    return array_to_img(aug_img)

def extract_features_from_paths(model, img_paths, model_name):
    features = []
    for img_path in img_paths:
        img = load_img(img_path, target_size=model_input_sizes[model_name])
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        preprocessed_img = model_info[model_name][1](img_array)
        feats = model.predict(preprocessed_img, verbose=0)
        features.append(feats.flatten())
    return features

def extract_features_augmented(model, img_paths, model_name, target_len=50):
    features = extract_features_from_paths(model, img_paths, model_name)
    needed = target_len - len(features)
    if needed > 0:
        with ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(augment_and_extract, model, random.choice(img_paths), model_name) for _ in range(needed)]
            for f in futures:
                features.append(f.result())
    return features[:target_len]

def run_stratified_kfold_regressions_from_images(df, model, model_name):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_results = []
    all_preds_txt = []

    models = [
        ('Linear Regression', LinearRegression()),
        ('Random Forest', RandomForestRegressor(random_state=42)),
        ('SVR', SVR()),
        ('Lasso', Lasso(random_state=42)),
        ('ElasticNet', ElasticNet(random_state=42)),
        ('Ridge', Ridge(random_state=42))
    ]

    for fold, (train_idx, test_idx) in enumerate(skf.split(df, df['Group'])):
        df_train = df.iloc[train_idx]
        df_test = df.iloc[test_idx]

        id_to_train_paths = defaultdict(list)
        id_to_test_paths = defaultdict(list)

        for _, row in df_train.iterrows():
            id_to_train_paths[row['ID Animal']].append(row['Path'])
        for _, row in df_test.iterrows():
            id_to_test_paths[row['ID Animal']].append(row['Path'])

        train_features, test_features = {}, {}
        for animal_id in tqdm(id_to_train_paths.keys(), desc=f"Fold {fold+1} - Treino ({model_name})"):
            paths = id_to_train_paths[animal_id]
            feats = extract_features_augmented(model, paths, model_name, target_len=50)
            train_features[animal_id] = np.mean(feats, axis=0)

        for animal_id in tqdm(id_to_test_paths.keys(), desc=f"Fold {fold+1} - Teste ({model_name})"):
            paths = id_to_test_paths[animal_id]
            feats = extract_features_from_paths(model, paths, model_name)
            test_features[animal_id] = np.mean(feats, axis=0)

        df_weights = df[['ID Animal', 'Peso', 'Group']].drop_duplicates().set_index('ID Animal')
        train_df = pd.DataFrame.from_dict(train_features, orient='index').join(df_weights)
        test_df = pd.DataFrame.from_dict(test_features, orient='index').join(df_weights)

        X_train = train_df.iloc[:, :-2].values
        y_train = train_df['Peso'].values
        X_test = test_df.iloc[:, :-2].values
        y_test = test_df['Peso'].values

        for name, reg_model in models:
            model_clone = clone(reg_model)
            model_clone.fit(X_train, y_train)
            y_pred = model_clone.predict(X_test)

            print(f"\n--- Regressão: {name} | Modelo CNN: {model_name} ---")
            fold_metrics = []

            for i, (real, pred) in enumerate(zip(y_test, y_pred)):
                animal_id = test_df.index[i]
                all_preds_txt.append((model_name, name, fold+1, animal_id, real, pred))

            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            r = pearsonr(y_test, y_pred)[0] if np.var(y_pred) > 0 else 0

            print(f"[Fold {fold+1}] MSE: {mse:.2f} | RMSE: {rmse:.2f} | MAE: {mae:.2f} | R2: {r2:.2f} | R: {r:.2f}")

            all_results.append({
                'REGRESSAO': name,
                'FOLD': fold + 1,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae,
                'R2': r2,
                'R': r,
                'CNN': model_name
            })

    lines = ["CNN,REGRESSAO,FOLD,ID,PESO_REAL,PESO_PREDITO"]
    for cnn, reg, fold, id_, real, pred in all_preds_txt:
        lines.append(f"{cnn},{reg},{fold},{id_},{real:.2f},{pred:.2f}")
    with open(f"predicoes_completas_{model_name}.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print(f"\n📝 Predições consolidadas salvas em predicoes_completas_{model_name}.txt")

    df_result = pd.DataFrame(all_results)
    for reg_name in df_result['REGRESSAO'].unique():
        df_model = df_result[df_result['REGRESSAO'] == reg_name]
        print(f"\n>>> MÉDIA DOS RESULTADOS - {reg_name}")
        for metric in ['MSE', 'RMSE', 'MAE', 'R2', 'R']:
            values = df_model[metric].values
            print(f"{metric}: {np.mean(values):.2f} ± {np.std(values):.2f}")

    df_result.to_excel(f"metricas_por_fold_{model_name}.xlsx", index=False)
    print(f"📊 Métricas por fold salvas em metricas_por_fold_{model_name}.xlsx")
    return all_results

# --- Execução principal ---
df = pd.read_csv('all_animal_images_T.csv')
df['Group'] = df['ID Animal'].apply(lambda x: 'C1' if 'C1_' in x else 'C2')

final_results = []
for model_name in model_info.keys():
    print(f"\nIniciando processamento com modelo: {model_name}")
    start_time = time.time()
    model = model_info[model_name][0](weights='imagenet', include_top=False, pooling='avg')
    results = run_stratified_kfold_regressions_from_images(df, model, model_name)
    final_results.extend(results)
    elapsed = time.time() - start_time
    print(f"Tempo total para {model_name}: {elapsed/60:.2f} minutos")

results_df = pd.DataFrame(final_results)
results_df.to_excel('regression_results_1024_ResNet50_augmented_trainonly_stratified.xlsx', index=False)
print("Resultados salvos em 'regression_results_1024_ResNet50_augmented_trainonly_stratified.xlsx'.")


Iniciando processamento com modelo: ResNet50


Fold 1 - Teste (ResNet50): 100%|███████████████████████████████████████████████████████| 72/72 [01:02<00:00,  1.15it/s]



--- Regressão: Linear Regression | Modelo CNN: ResNet50 ---
[Fold 1] MSE: 22.65 | RMSE: 4.76 | MAE: 3.29 | R2: 0.83 | R: 0.95

--- Regressão: Random Forest | Modelo CNN: ResNet50 ---
[Fold 1] MSE: 70.34 | RMSE: 8.39 | MAE: 6.57 | R2: 0.47 | R: 0.89

--- Regressão: SVR | Modelo CNN: ResNet50 ---
[Fold 1] MSE: 126.62 | RMSE: 11.25 | MAE: 9.78 | R2: 0.05 | R: 0.81

--- Regressão: Lasso | Modelo CNN: ResNet50 ---
[Fold 1] MSE: 92.33 | RMSE: 9.61 | MAE: 7.29 | R2: 0.31 | R: 0.77

--- Regressão: ElasticNet | Modelo CNN: ResNet50 ---
[Fold 1] MSE: 85.65 | RMSE: 9.25 | MAE: 7.16 | R2: 0.36 | R: 0.81

--- Regressão: Ridge | Modelo CNN: ResNet50 ---
[Fold 1] MSE: 16.32 | RMSE: 4.04 | MAE: 3.16 | R2: 0.88 | R: 0.96


Fold 2 - Teste (ResNet50): 100%|███████████████████████████████████████████████████████| 71/71 [01:30<00:00,  1.28s/it]



--- Regressão: Linear Regression | Modelo CNN: ResNet50 ---
[Fold 2] MSE: 13.76 | RMSE: 3.71 | MAE: 2.99 | R2: 0.89 | R: 0.95

--- Regressão: Random Forest | Modelo CNN: ResNet50 ---
[Fold 2] MSE: 68.61 | RMSE: 8.28 | MAE: 6.60 | R2: 0.47 | R: 0.91

--- Regressão: SVR | Modelo CNN: ResNet50 ---
[Fold 2] MSE: 126.45 | RMSE: 11.25 | MAE: 9.83 | R2: 0.02 | R: 0.78

--- Regressão: Lasso | Modelo CNN: ResNet50 ---
[Fold 2] MSE: 83.21 | RMSE: 9.12 | MAE: 6.68 | R2: 0.35 | R: 0.78

--- Regressão: ElasticNet | Modelo CNN: ResNet50 ---
[Fold 2] MSE: 81.26 | RMSE: 9.01 | MAE: 6.64 | R2: 0.37 | R: 0.80

--- Regressão: Ridge | Modelo CNN: ResNet50 ---
[Fold 2] MSE: 12.08 | RMSE: 3.48 | MAE: 2.71 | R2: 0.91 | R: 0.95


...


--- Regressão: Linear Regression | Modelo CNN: ResNet50 ---
[Fold 3] MSE: 17.72 | RMSE: 4.21 | MAE: 3.37 | R2: 0.84 | R: 0.92

--- Regressão: Random Forest | Modelo CNN: ResNet50 ---
[Fold 3] MSE: 25.90 | RMSE: 5.09 | MAE: 3.90 | R2: 0.77 | R: 0.92

--- Regressão: SVR | Modelo CNN: ResNet50 ---
[Fold 3] MSE: 101.58 | RMSE: 10.08 | MAE: 8.94 | R2: 0.08 | R: 0.79

--- Regressão: Lasso | Modelo CNN: ResNet50 ---
[Fold 3] MSE: 51.37 | RMSE: 7.17 | MAE: 5.58 | R2: 0.54 | R: 0.79

--- Regressão: ElasticNet | Modelo CNN: ResNet50 ---
[Fold 3] MSE: 50.99 | RMSE: 7.14 | MAE: 5.60 | R2: 0.54 | R: 0.80

--- Regressão: Ridge | Modelo CNN: ResNet50 ---
[Fold 3] MSE: 13.03 | RMSE: 3.61 | MAE: 2.80 | R2: 0.88 | R: 0.94


Fold 4 - Teste (ResNet50): 100%|███████████████████████████████████████████████████████| 70/70 [01:03<00:00,  1.10it/s]



--- Regressão: Linear Regression | Modelo CNN: ResNet50 ---
[Fold 4] MSE: 14.31 | RMSE: 3.78 | MAE: 2.66 | R2: 0.85 | R: 0.94

--- Regressão: Random Forest | Modelo CNN: ResNet50 ---
[Fold 4] MSE: 24.48 | RMSE: 4.95 | MAE: 3.91 | R2: 0.75 | R: 0.91

--- Regressão: SVR | Modelo CNN: ResNet50 ---
[Fold 4] MSE: 90.50 | RMSE: 9.51 | MAE: 8.29 | R2: 0.07 | R: 0.74

--- Regressão: Lasso | Modelo CNN: ResNet50 ---
[Fold 4] MSE: 51.24 | RMSE: 7.16 | MAE: 5.67 | R2: 0.47 | R: 0.75

--- Regressão: ElasticNet | Modelo CNN: ResNet50 ---
[Fold 4] MSE: 49.79 | RMSE: 7.06 | MAE: 5.60 | R2: 0.49 | R: 0.77

--- Regressão: Ridge | Modelo CNN: ResNet50 ---
[Fold 4] MSE: 11.71 | RMSE: 3.42 | MAE: 2.39 | R2: 0.88 | R: 0.95


Fold 5 - Teste (ResNet50): 100%|███████████████████████████████████████████████████████| 76/76 [01:05<00:00,  1.16it/s]



--- Regressão: Linear Regression | Modelo CNN: ResNet50 ---
[Fold 5] MSE: 14.08 | RMSE: 3.75 | MAE: 2.72 | R2: 0.87 | R: 0.93

--- Regressão: Random Forest | Modelo CNN: ResNet50 ---
[Fold 5] MSE: 87.90 | RMSE: 9.38 | MAE: 7.45 | R2: 0.21 | R: 0.81

--- Regressão: SVR | Modelo CNN: ResNet50 ---
[Fold 5] MSE: 106.45 | RMSE: 10.32 | MAE: 9.31 | R2: 0.04 | R: 0.77

--- Regressão: Lasso | Modelo CNN: ResNet50 ---
[Fold 5] MSE: 85.16 | RMSE: 9.23 | MAE: 7.08 | R2: 0.23 | R: 0.76

--- Regressão: ElasticNet | Modelo CNN: ResNet50 ---
[Fold 5] MSE: 82.06 | RMSE: 9.06 | MAE: 7.04 | R2: 0.26 | R: 0.78

--- Regressão: Ridge | Modelo CNN: ResNet50 ---
[Fold 5] MSE: 10.65 | RMSE: 3.26 | MAE: 2.50 | R2: 0.90 | R: 0.95

📝 Predições consolidadas salvas em predicoes_completas_ResNet50.txt

>>> MÉDIA DOS RESULTADOS - Linear Regression
MSE: 16.50 ± 3.39
RMSE: 4.04 ± 0.40
MAE: 3.01 ± 0.29
R2: 0.86 ± 0.02
R: 0.94 ± 0.01

>>> MÉDIA DOS RESULTADOS - Random Forest
MSE: 55.44 ± 25.61
RMSE: 7.22 ± 1.84
MAE: 5.